# Learning Machine

The objective of this experiment is to build a learning machine. Specifically it should have the ability to 

Vision
-----------------
1. Given any input, give a text output of what it think it is. e.g. given an input of image of square it should say square.
2. Given a text input , give out the image of required output. e.g. if input is "square" give output an image that looks like a sqare.
3. Answer simple questions like google now


Technically it will be like this. 

Have a bunch of classifiers (Imagenet CNN, speech recognizing CNN ... etc). A bank of classifiers.
And an intelligent way to manage these classifiers. It should know when to call (predict, train and modify these classifiers).

Some questions. Are concepts tied to a particular classifier or to a particular pattern of data. e.g. an input image of dog is given. Is there a pattern generated that is same for every time a dog picture is given? or is there a classifier that is on whenver there is a dog image. The former can trigger a latter. It is trivial to create a classifier that detects known pattern. Then the question is now to generate this invariant pattern that should be generate whenever a dog image is shown. 
Let us think for a second how this would work. There are some things that are mostly there. Like fur, two eyes, tail, four legs etc. 

Side track
==============

Let me first try building this infinite recursive learner thing.

In [2]:
import numpy as np
from sklearn import svm

In [113]:
maximum_input_dimension = 1e4

class inf_rec_learner:
    """ The infinite recursive learner which keeps adding layers upon layers.
    API should be similar to scikit learn"""
    
    def __init__(self, classifiers_list = None):
        if classifiers_list == None:
            self.classifiers_list = []
        else:
            self.classifiers_list = classifiers_list
        self.number_of_classifier = len(self.classifiers_list)
        
    def fit(self, X,y):
        # let's see what the existing classifier have to say about the incoming data.
        X = np.array(X)
        print X.shape
        in_sample_size = X.shape[0]
        if len(X.shape) > 1:
            in_dim = X.shape[1] 
        # Assumption, single output, each prediction is a column in the matrix, stack them horizontally 
        # to build a matrix
        if self.classifiers_list == []:
            self.side_info = []
        else:
            self.side_info = self.predict(X)
        # side info should be a tensorr matrix with dimension (number of classifiers, X rows, variable width)
        print 'Side info'
        print self.side_info
        if len(self.side_info) == 0:
            new_X = X 
        else:
            new_X = np.vstack([np.hstack([X[j,:], self.side_info[j,:]]) for j in range(in_sample_size)])
        print 'The new X is '
        print new_X
        clf = svm.SVC()
        clf.fit(new_X,y)
        self.classifiers_list.append(clf)
        self.number_of_classifier = len(self.classifiers_list)
    
    def predict(self, X):
        """Give out what it thinks from the input."""
        return np.hstack([np.array(i.predict(X)).reshape(-1,1) for i in self.classifiers_list]) 
        
    def status(self):
        """Gives out the current status, like number of classifier and prints their values"""
        print 'Currently there are ', len(self.classifiers_list), ' classifiers. They are'
        for classifier_i in self.classifiers_list:
            print classifier_i

In [114]:
rab1 = inf_rec_learner()
X = [[0, 0], [1, 1]]
y = [0, 1]

In [118]:
rab1.fit(X,y)

(2L, 2L)


ValueError: X.shape[1] = 2 should be equal to 3, the number of features at training time

In [119]:
rab1.status()

Currently there are  2  classifiers. They are
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


In [87]:
rab1.side_info

array([], shape=(0L, 1L), dtype=float64)

In [77]:
X = np.array(X)
X.shape

(2L, 2L)

In [98]:
rab1.predict(X).shape

(0L,)

In [91]:
np.hstack([  np.array([1,2,3]).reshape(-1,1),  np.array([ 2,3,4]).reshape(-1,1)   ])

array([[1, 2],
       [2, 3],
       [3, 4]])

In [99]:
rab1.classifiers_list

[]

In [101]:
np.hstack([])

IndexError: list index out of range

**Problem** for predict algorithm
the input data is different shape than the one trained with, how to determine which classifier to use and which not? Need a gating mech

In [6]:
big_matrix = np.random.randn(5,5)
print big_matrix

[[ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]
 [-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]
 [-1.34605532 -0.09470203 -1.06488969 -0.55580451  0.79569981]
 [-0.36516623  0.4704561  -0.27349477  0.12732577 -0.14797002]
 [-0.96902526 -0.25813108 -1.16039883  0.31218969  0.4037262 ]]


In [ ]:
X = np.zeros([2,3])

In [17]:
print big_matrix[1:X.shape[0],1:X.shape[1]]

[[-0.7679232  0.4578361]]


In [20]:
print np.where(1,big_matrix,X)

ValueError: operands could not be broadcast together with shapes () (5,5) (2,3) 

In [22]:
print big_matrix[np.indices(X.shape)]

[[[[ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]
   [ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]
   [ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]]

  [[-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]
   [-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]
   [-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]]]


 [[[ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]
   [-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]
   [-1.34605532 -0.09470203 -1.06488969 -0.55580451  0.79569981]]

  [[ 0.52418615  0.42610001  0.8083232   0.48716289 -0.51250622]
   [-0.16307508 -0.7679232   0.4578361  -1.84949592  0.1304195 ]
   [-1.34605532 -0.09470203 -1.06488969 -0.55580451  0.79569981]]]]


In [24]:
print np.mgrdi(5,2)

AttributeError: 'module' object has no attribute 'mgrdi'